# <b> Chapter 6

In [45]:
import enum
import random

TIMES = 10_000
random.seed(23)

In [46]:
def rounder(val=2):
    def outer(fn):
        def inner(*args, **kwargs):
            res = fn(*args, **kwargs)
            return round(res, val)
        return inner
    return outer

## Conditional Probability

In [48]:
# Calculating the probablity of children genders

# No girls -> 1/2 * 1/2 = 1/4
# Two girls -> 1/2 * 1/2 = 1/4. Same as above.
# One girl, one boy (vice-versa) -> 1/2 * 1/2 + 1/2 * 1/2 = 1/2.
# Both children are girls, given that at least one of them is a girl (conditional). 1/2 or 1/3 (two scenarios)


class Child(enum.IntEnum):
    BOY = 0
    GIRL = 1


def rand_child() -> Child:
    return random.choice([Child.BOY, Child.GIRL])


# The first has to be a boy as well as the second one
@rounder()
def no_girls() -> float:
    boy = 0
    girl = 0
    for _ in range(TIMES):
        if rand_child() == Child.BOY:
            boy += 1
        else:
            girl += 1
    # Probability of a boy given all scenarios.
    first_child = (boy / (boy + girl))
    return first_child * first_child  # Independent events


# Same as above
@rounder()
def two_girls() -> float:
    boy = 0
    girl = 0
    for _ in range(TIMES):
        if rand_child() == Child.BOY:
            boy += 1
        else:
            girl += 1
    # Probability of a boy given all scenarios.
    first_child = (girl / (boy + girl))
    return first_child * first_child  # Independent events


# The first can either be a girl or a boy (1/2*1/2), but the second must be a boy or a girl in that order.
@rounder()
def girl_boy() -> float:
    boy = 0
    girl = 0
    for _ in range(TIMES):
        first_child = rand_child()
        second_child = rand_child()
        if first_child == Child.BOY:
            boy += 1
            if second_child == Child.GIRL:
                girl += 1
        if first_child == Child.GIRL:
            girl += 1
            if second_child == Child.BOY:
                boy += 1

    boy_girl = (boy / (boy+girl)) * (girl / (boy + girl))  # Boy and girl
    girl_boy = (girl / (boy + girl)) * (boy / (boy+girl))  # Girl and boy
    return boy_girl + girl_boy

@rounder()
def girl_girl(mol=0) -> float:
    both_girls = 0
    older_girl = 0
    either_girl = 0
    for _ in range(TIMES):
        first_child = rand_child()
        second_child = rand_child()
        if first_child == Child.GIRL:
            older_girl += 1
        if first_child == Child.GIRL and second_child == Child.GIRL:
            both_girls += 1
        if first_child == Child.GIRL or second_child == Child.GIRL:
            either_girl += 1
    match mol:
        case 0: return both_girls / older_girl
        case 1: return  both_girls / either_girl


print(f'P(B) = {no_girls()}\nP(G) = {two_girls()}\nP(G, B) = {girl_boy()}\nP(B | L) = {girl_girl(0)}\nP(B | G) = {girl_girl(1)}')

P(B) = 0.25
P(G) = 0.25
P(G, B) = 0.5
P(B | L) = 0.5
P(B | G) = 0.34
